In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import TfidfVectorizer

# Read recipe inputs
convers_validation = dataiku.Dataset("convers_validation")
convers_validation_df = convers_validation.get_dataframe()
convers_validation_df = convers_validation_df.drop_duplicates(subset='nom', keep=False) #supprimer les doublons dans la colonne 'nom'
convers_satisfac_df = convers_validation_df
#récupérer les mots dans le dictionnaire LIWCD
mot_anxiogene = dataiku.Dataset("mot_anxi")
mot_anxiogene_df = mot_anxiogene.get_dataframe()

mot_colere = dataiku.Dataset("mots_colere")
mot_colere_df = mot_colere.get_dataframe()

mot_tristesse = dataiku.Dataset("mots_tristesse")
mot_tristesse_df = mot_tristesse.get_dataframe()

mot_divergence = dataiku.Dataset("mots_divergence")
mot_divergence_df = mot_divergence.get_dataframe()

mot_travail = dataiku.Dataset("mots_travail")
mot_travail_df = mot_travail.get_dataframe()

mot_hesitation = dataiku.Dataset("mots_hesitation")
mot_hesitation_df = mot_hesitation.get_dataframe()

mot_exclusion = dataiku.Dataset("mots_exclusion")
mot_exclusion_df = mot_exclusion.get_dataframe()

In [ ]:
#convers_all = convers_validation_df['text_diarise']
convers_all = convers_validation_df['text_operateur']
id_conv = convers_validation_df['nom']
liste_id_conv = list(id_conv)
liste_convers_all = list(convers_all)
mot_anxio = mot_anxiogene_df['mot']
satis = convers_validation_df['SATISFACTION_GLOBALE']
validation = convers_validation_df['eva_validation']
liste_satis = list(satis)
liste_validation = list(validation)


liste_mot_anxio_all = list(mot_anxio)
liste_mot_colere_all = list(mot_colere_df['mot'])
liste_mot_tristesse_all = list(mot_tristesse_df['mot'])
liste_mot_divergence_all = list(mot_divergence_df['mot'])
liste_mot_travail_all = list(mot_travail_df['mot'])
liste_mot_hesitation_all = list(mot_hesitation_df['mot'])
liste_mot_exclusion_all = list(mot_exclusion_df['mot'])

liste_mot_total = liste_mot_anxio_all+liste_mot_colere_all+liste_mot_tristesse_all+liste_mot_divergence_all+liste_mot_travail_all+liste_mot_hesitation_all+liste_mot_exclusion_all

In [ ]:
#supprimer les * à la fin de mot et ajouter une espace avant de mot (pour éviter le problème : chercher 'tendu', trouver: attendu)
def liste_lemme(liste):
    liste_mot = []
    liste_mot_final = []
    for m in liste:
        if "*" in m:
            m = m[:-1]
            liste_mot.append(m)
        else:
            liste_mot.append(m)

    for mot in liste_mot:
        mot_final = ' '+mot
        liste_mot_final.append(mot_final)

    return liste_mot_final

In [ ]:
import re
def detection_mots(liste):
    
    liste_all = []

    for i in range(len(liste_convers_all)):
        conv = liste_convers_all[i]
        
        dic_conv = {}
        dic_conv['nom'] = liste_id_conv[i]
        liste_mot = []

        for mot in liste:
            if mot in conv:                    # si le mot dans la conversation




                    #freq = liste_convers_all[i].count(mot)        #trouver la présence et la fréquance
                                                                    #enlever l'espace devant le mot
                    regex = re.compile(str(mot)+r'\w*')            #trouver le mot au lieu de lemme
                    mot_trouve = regex.search(conv)
                    if mot_trouve:
                        #dic_mot = {}
                        mot_trouve = mot_trouve.group()
                        mot_trouve = mot_trouve[1:]
                        #freq = liste_convers_all[i].count(mot_trouve)

                        liste_mot.append(mot_trouve)

        dic_conv['liste_nom'] = liste_mot
        dic_conv['nbr_mot'] = len(liste_mot)
        liste_all.append(dic_conv)

                    #dic_mot['token'] = mot_trouve
                    

    return liste_all

In [ ]:
'''
liste_mot = []
for i in range(len(liste_convers_all)):
    dic_mot = {}
    id_conv = liste_id_conv[i]
    liste = liste_convers_all[i].split(' ')
    nbr_token = len(liste)
    satis = liste_satis[i]
    validation = liste_validation[i]
    len_anxio = len(detection_mots_anxio(liste_lemme(liste_mot_anxio_all[i])))
    len_colere = len(detection_mots_colere(liste_lemme(liste_mot_colere_all[i])))
    len_tristesse = len(detection_mots_tristesse(liste_lemme(liste_mot_tristesse_all[i])))
    len_travail = len(detection_mots_travail(liste_lemme(liste_mot_travail_all[i])))
    len_divergence = len(detection_mots_divergence(liste_lemme(liste_mot_divergence_all[i])))
    #len_hesitation = len(detection_mots_hesitation(liste_lemme(liste_mot_hesitation_all[i])))
    len_exclusion = len(detection_mots_exclusion(liste_lemme(liste_mot_exclusion_all[i])))
    len_mot = len_anxio+len_colere+len_tristesse+len_travail+len_divergence+len_exclusion#+len_hesitation
    dic_mot['nom'] = id_conv
    dic_mot['nbr_token'] = nbr_token
    dic_mot['nbr_mot_anxio'] = len_mot
    dic_mot['SATISFACTION_GLOBALE'] = satis
    dic_mot['eva_validation'] = validation
    liste_mot.append(dic_mot)
'''

In [ ]:
new_df = pd.DataFrame( detection_mots(liste_lemme(liste_mot_total)))
convers_satisfac = dataiku.Dataset("token_mot_convers")
convers_satisfac.write_with_schema(new_df)